In [7]:
import pygame
import numpy as np
import math

class Node:
    def __init__(self, x, y, vars=None):
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (e.g., temperature, etc.)

    def update(self):
        # Updates all internal variables (if any) as required
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draws the node on the screen
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)


def construction(x_range, y_range, spacing):
    """ 
    Creates an array of nodes within the specified x and y ranges, 
    with a defined distance between all nodes.
    """
    environment = []
    i = 0
    while x_range[0] + i * spacing <= x_range[1]:
        j = 0
        while y_range[0] + j * spacing <= y_range[1]:
            var_specific = None  # Modify this as needed for custom variables
            environment.append(Node(x_range[0] + i * spacing, y_range[0] + j * spacing, var_specific))
            j += 1
        i += 1
    return environment


def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
    """ Generate a finer grid around a selected node within a circular region """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    radius_squared = radius ** 2

    # Define the range to search for points in the circular region
    range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
    range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

    for i in range_x:
        for j in range_y:
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Calculate distance from the selected node
            distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

            # Check if the point is within the circular region and within the coarse boundary
            if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                nodes.append(Node(x, y))
    return nodes


def generate_fine_grid(selected_node, range_units, fine_spacing, coarse_max_x, coarse_max_y):
    """ Generate a regular finer grid around a selected node """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    for i in range(int(range_units / fine_spacing) + 1):
        for j in range(int(range_units / fine_spacing) + 1):
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Check to ensure the fine grid does not exceed the coarse grid boundaries
            if x <= coarse_max_x and y <= coarse_max_y:
                nodes.append(Node(x, y))
    return nodes


# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Conditional Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60  # Set a higher FPS for smoother rendering

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1  # Coarse grid spacing (1 unit)

# Calculate dynamic scaling factor so that the grid fits within the window
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])  # 100 pixels margin
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])  # 100 pixels margin
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Generate the coarse grid
coarse_nodes = construction(x_range, y_range, spacing)

# Define the minimum and maximum boundary values for the coarse grid
min_x, min_y = coarse_nodes[0].position  # Top-left corner of the grid
max_x, max_y = coarse_nodes[-1].position  # Bottom-right corner of the grid

# Function to refine based on the node and refinement type
def refine_grid(selected_node, refine_type, fine_spacing, radius, range_units, min_x, max_x, min_y, max_y):
    if refine_type == 'circular':
        return generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y)
    elif refine_type == 'regular':
        return generate_fine_grid(selected_node, range_units, fine_spacing, max_x, max_y)
    else:
        return []

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

fine_nodes = []
for node, refine_type in nodes_to_refine:
    fine_nodes.extend(refine_grid(node, refine_type, fine_spacing=0.5, radius=2, range_units=2, min_x=min_x, max_x=max_x, min_y=min_y, max_y=max_y))

# Main loop
running = True

while running:
    window.fill((255, 255, 255))  # Clear screen with a simple white background

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw all coarse grid nodes
    for node in coarse_nodes:
        node.draw(window, (0, 0, 255), scaling_factor, offset_x, offset_y)  # Coarse grid nodes in blue

    # Draw all fine grid nodes (refined nodes)
    for node in fine_nodes:
        node.draw(window, (255, 0, 0), scaling_factor, offset_x, offset_y)  # Fine grid nodes in red

    pygame.display.flip()  # Update the screen
    clock.tick(fps)  # Maintain the frame rate

# Quit Pygame
pygame.quit()


In [6]:
import pygame
import numpy as np
import math

class Node:
    def __init__(self, x, y, vars=None):
        self.position = np.array([float(x), float(y)])  # Position of the node
        self.vars = vars  # Any additional variables for each node (optional)

    def update(self):
        # Update logic for the node (if needed)
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y, pygame):
        # Draw the node (grid point)
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)

def construction(x_range, y_range, spacing, refine_list=None, fine_spacing=0.5, radius=2, range_units=2):
    """
    Creates an array of nodes (environment) within the specified x and y ranges, with spacing.
    It also allows for refinement around specific nodes (coarse or circular).
    """
    environment = []
    for i in np.arange(x_range[0], x_range[1], spacing):
        for j in np.arange(y_range[0], y_range[1], spacing):
            node = Node(i, j)
            environment.append(node)

    # Refine specific nodes if refine_list is provided
    if refine_list:
        fine_nodes = []
        for node, refine_type in refine_list:
            if refine_type == 'circular':
                fine_nodes.extend(generate_fine_grid_circular(node, fine_spacing, radius, x_range[0], x_range[1], y_range[0], y_range[1]))
            elif refine_type == 'regular':
                fine_nodes.extend(generate_fine_grid(node, range_units, fine_spacing, x_range[1], y_range[1]))
        environment.extend(fine_nodes)  # Add refined nodes to the environment

    return environment

def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
    """ Generate a finer grid around a selected node within a circular region """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    radius_squared = radius ** 2

    range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
    range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

    for i in range_x:
        for j in range_y:
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

            if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                nodes.append(Node(x, y))
    return nodes

def generate_fine_grid(selected_node, range_units, fine_spacing, max_x, max_y):
    """ Generate a regular finer grid around a selected node """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    for i in range(int(range_units / fine_spacing) + 1):
        for j in range(int(range_units / fine_spacing) + 1):
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing

            if x <= max_x and y <= max_y:
                nodes.append(Node(x, y))
    return nodes

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Environment Grid Visualization")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60

# Environment setup (creating nodes in the environment)
x_range = [0, 10]
y_range = [0, 10]
spacing = 1  # Coarse grid spacing

# First, generate the coarse environment
env_list = construction(x_range, y_range, spacing)

# Define nodes to refine and their respective refinement types AFTER creating the coarse grid
refine_list = [
    (env_list[0], 'circular'),  # Refine around node 0 circularly
    (env_list[5], 'regular'),   # Refine around node 5 regularly
]

# Regenerate the environment with fine refinements
env_list = construction(x_range, y_range, spacing, refine_list)

# Main loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a white background

    # Update and draw the environment (nodes)
    for point in env_list:
        point.update()
        point.draw(window, (0, 0, 255), 5, 50, 50, pygame)  # Blue for nodes

    pygame.display.flip()  # Update the display
    clock.tick(fps)  # Limit the frame rate

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Quit Pygame
pygame.quit()


In [ ]:
import math
import random
import numpy as np

import environment as env
from agent import Agent






#creation of environmental list
x_range = [x_min, x_max]
y_range = [y_min, y_max]
spacing  = 10 # all nodes are spaced 10 units apart from each other
env_list = env.construction(x_range,y_range,spacing)

#creation of list of all agents
ag_list = []
att = att
var = var
obj = obj #define attributes, variables, and objective for each agent individually
ag_list.append(Agent(att, var, obj))
# due to the fact that agents are both much less prolific than nodes, and often require specific
# initial placement, each agent is append to the list seperately

continuing = True
while continuing:
    #go through all bots and nodes, updating to the next time step
    for bot in ag_list:
        bot.sensing(env_list, ag_list)
        bot.processing()
    for point in env_list:
        point.update()
    for bot in ag_list:
        bot.actuating()

    if conditions == True:
        continuing = False
        #if certain conditions are met, end the loop running the simulation


# if rendering in post , a copy of each agents self.X and the variables of each node could be saved at
# each time step, and exported to be run in a seperate function.
    



In [ ]:
import math
import random
import numpy as np

## Contains the agent class, used to represent self contained entitites utilizing swarm algorithms
class Agent:
    def __init__(self, attributes, variables, objective):
        # Atrributes are used to represent external aspects of the agent, such as its size,
        # position, and any aspects that can be directly observed
        self.X = attributes
        # Variables are used to represent internal aspects of the agent, including memory or pressure
        self.V = variables
        # Objectives are used to represent the goals of the agent, such as a target position
        self.O = objective
        # the local environment is found using the sensors of the agent, and includes all nearby nodes
        # from the global environment and all nearby agents
        # as the local environment is defined at each time step, this variable is left blank during
        # initialization
        node_info = []
        agent_info = []
        self.E = [node_info,agent_info]

    def sensing(self, environment, agents):
        # using the global list of all environmental information and all other agents, update self.E
        # to contain a list of only the environmental and other agent's information that the agent
        # is able to detect. constraints on sensing would typically be found under self.X
        self.E = [environment, agents]
        # in this example function, the agent is able to detect all environmental and agent information
        # in the simulation

    def processing(self):
        # using its information self.X,V,O, and E, the agent updates its internal information
        # based on its internal functions self.V and its objectives self.O
        pass #add code here depending on what is being changed

    def actuating(self):
        # using its information, the agent updates its external information self.X
        pass #add code here depending on what is being changed

In [9]:
import pygame
import numpy as np
import math

class Node:
    def __init__(self, x, y, vars=None):
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (e.g., temperature, etc.)

    def update(self):
        # Updates all internal variables (if any) as required
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draws the node on the screen
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)

class Agent:
    def __init__(self, x, y):
        self.position = np.array([float(x), float(y)])  # x, y position of the agent
        self.size = 10  # Size of the agent

    def update(self):
        # Update logic for the agent (if needed)
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the agent on the screen
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           self.size)

def construction(x_range, y_range, spacing):
    """ 
    Creates an array of nodes within the specified x and y ranges, 
    with a defined distance between all nodes.
    """
    environment = []
    i = 0
    while x_range[0] + i * spacing <= x_range[1]:
        j = 0
        while y_range[0] + j * spacing <= y_range[1]:
            var_specific = None  # Modify this as needed for custom variables
            environment.append(Node(x_range[0] + i * spacing, y_range[0] + j * spacing, var_specific))
            j += 1
        i += 1
    return environment


def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
    """ Generate a finer grid around a selected node within a circular region """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    radius_squared = radius ** 2

    # Define the range to search for points in the circular region
    range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
    range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

    for i in range_x:
        for j in range_y:
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Calculate distance from the selected node
            distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

            # Check if the point is within the circular region and within the coarse boundary
            if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                nodes.append(Node(x, y))
    return nodes


def generate_fine_grid(selected_node, range_units, fine_spacing, coarse_max_x, coarse_max_y):
    """ Generate a regular finer grid around a selected node """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    for i in range(int(range_units / fine_spacing) + 1):
        for j in range(int(range_units / fine_spacing) + 1):
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Check to ensure the fine grid does not exceed the coarse grid boundaries
            if x <= coarse_max_x and y <= coarse_max_y:
                nodes.append(Node(x, y))
    return nodes


# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60  # Set a higher FPS for smoother rendering

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1  # Coarse grid spacing (1 unit)

# Calculate dynamic scaling factor so that the grid fits within the window
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])  # 100 pixels margin
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])  # 100 pixels margin
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Generate the coarse grid
coarse_nodes = construction(x_range, y_range, spacing)

# Define the minimum and maximum boundary values for the coarse grid
min_x, min_y = coarse_nodes[0].position  # Top-left corner of the grid
max_x, max_y = coarse_nodes[-1].position  # Bottom-right corner of the grid

# Function to refine based on the node and refinement type
def refine_grid(selected_node, refine_type, fine_spacing, radius, range_units, min_x, max_x, min_y, max_y):
    if refine_type == 'circular':
        return generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y)
    elif refine_type == 'regular':
        return generate_fine_grid(selected_node, range_units, fine_spacing, max_x, max_y)
    else:
        return []

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

fine_nodes = []
for node, refine_type in nodes_to_refine:
    fine_nodes.extend(refine_grid(node, refine_type, fine_spacing=0.5, radius=2, range_units=2, min_x=min_x, max_x=max_x, min_y=min_y, max_y=max_y))

# Agent initialization (add agents at specific positions)
agents = [
    Agent(2, 3),  # Agent at position (2, 3)
    Agent(6, 5),  # Agent at position (6, 5)
    Agent(8, 9)   # Agent at position (8, 9)
]

# Main loop
running = True

while running:
    window.fill((255, 255, 255))  # Clear screen with a simple white background

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Draw all coarse grid nodes
    for node in coarse_nodes:
        node.draw(window, (0, 0, 255), scaling_factor, offset_x, offset_y)  # Coarse grid nodes in blue

    # Draw all fine grid nodes (refined nodes)
    for node in fine_nodes:
        node.draw(window, (255, 0, 0), scaling_factor, offset_x, offset_y)  # Fine grid nodes in red

    # Draw the agents
    for agent in agents:
        agent.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Agents in green

    pygame.display.flip()  # Update the screen
    clock.tick(fps)  # Maintain the frame rate

# Quit Pygame
pygame.quit()


In [3]:
import pygame
import numpy as np
import math

class Node:
    def __init__(self, x, y, vars=None):
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (optional)

    def update(self):
        # Update logic for the node (if needed)
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the node (grid point)
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)

class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = attributes  # External aspects (e.g., position, size)
        self.V = variables   # Internal aspects (e.g., memory, pressure)
        self.O = objective   # Objectives (e.g., target position)
        self.E = [[], []]    # Local environment (nodes and agents)

    def sensing(self, environment, agents):
        # Senses the local environment (e.g., nodes and agents in proximity)
        self.E = [environment, agents]

    def processing(self):
        # Processes the sensed data and updates internal state
        pass

    def actuating(self):
        # Updates the external state (e.g., position) based on internal state
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the agent on the screen
        pygame.draw.circle(surface, color,
                           (int(self.X[0] * scale_factor + offset_x),
                            int(self.X[1] * scale_factor + offset_y)),
                           10)  # Agents drawn as larger circles

def construction(x_range, y_range, spacing):
    """ 
    Creates an array of nodes within the specified x and y ranges, 
    with a defined distance between all nodes.
    """
    environment = []
    i = 0
    while x_range[0] + i * spacing <= x_range[1]:
        j = 0
        while y_range[0] + j * spacing <= y_range[1]:
            var_specific = None  # Modify this as needed for custom variables
            environment.append(Node(x_range[0] + i * spacing, y_range[0] + j * spacing, var_specific))
            j += 1
        i += 1
    return environment

def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
    """ Generate a finer grid around a selected node within a circular region """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    radius_squared = radius ** 2

    # Define the range to search for points in the circular region
    range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
    range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

    for i in range_x:
        for j in range_y:
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Calculate distance from the selected node
            distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

            # Check if the point is within the circular region and within the coarse boundary
            if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                nodes.append(Node(x, y))
    return nodes

def generate_fine_grid(selected_node, range_units, fine_spacing, coarse_max_x, coarse_max_y):
    """ Generate a regular finer grid around a selected node """
    nodes = []
    selected_node_x, selected_node_y = selected_node.position
    for i in range(int(range_units / fine_spacing) + 1):
        for j in range(int(range_units / fine_spacing) + 1):
            x = selected_node_x + j * fine_spacing
            y = selected_node_y + i * fine_spacing
            # Check to ensure the fine grid does not exceed the coarse grid boundaries
            if x <= coarse_max_x and y <= coarse_max_y:
                nodes.append(Node(x, y))
    return nodes

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents and Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60  # Set a higher FPS for smoother rendering

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1  # Coarse grid spacing (1 unit)

# Calculate dynamic scaling factor so that the grid fits within the window
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])  # 100 pixels margin
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])  # 100 pixels margin
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Generate the coarse grid
coarse_nodes = construction(x_range, y_range, spacing)

# Define the minimum and maximum boundary values for the coarse grid
min_x, min_y = coarse_nodes[0].position  # Top-left corner of the grid
max_x, max_y = coarse_nodes[-1].position  # Bottom-right corner of the grid

# Function to refine based on the node and refinement type
def refine_grid(selected_node, refine_type, fine_spacing, radius, range_units, min_x, max_x, min_y, max_y):
    if refine_type == 'circular':
        return generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y)
    elif refine_type == 'regular':
        return generate_fine_grid(selected_node, range_units, fine_spacing, max_x, max_y)
    else:
        return []

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

fine_nodes = []
for node, refine_type in nodes_to_refine:
    fine_nodes.extend(refine_grid(node, refine_type, fine_spacing=0.5, radius=2, range_units=2, min_x=min_x, max_x=max_x, min_y=min_y, max_y=max_y))

# Agent initialization (define agent attributes, variables, and objectives)
agents = [
    Agent([2.5, 3.5], {"memory": 0}, {"target": [5, 5]}),  # Example agent at position (2, 3)
    Agent([6.5, 5.5], {"memory": 0}, {"target": [9, 9]}),  # Example agent at position (6, 5)
    Agent([8.5, 9.5], {"memory": 0}, {"target": [10, 10]}) # Example agent at position (8, 9)
]

# Main simulation loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a simple white background

    # Go through all bots and nodes, updating to the next time step
    for bot in agents:
        bot.sensing(coarse_nodes, agents)  # Agent senses environment
        bot.processing()  # Agent processes information

    for point in coarse_nodes:
        point.update()  # Update the environment (nodes)
        point.draw(window, (0, 0, 255), scaling_factor, offset_x, offset_y)  # Blue nodes

    for point in fine_nodes:
        point.draw(window, (255, 0, 0), scaling_factor, offset_x, offset_y)  # Red refined nodes

    for bot in agents:
        bot.actuating()  # Agent actuates based on processing
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Green agents

    pygame.display.flip()  # Update the screen
    clock.tick(fps)  # Maintain the frame rate

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Quit Pygame
pygame.quit()


In [1]:
import math
import random
import numpy as np

import environment as env
from agent import Agent

#creation of environmental list
x_range = [x_min, x_max]
y_range = [y_min, y_max]
spacing  = 10 # all nodes are spaced 10 units apart from each other
env_list = env.construction(x_range,y_range,spacing)

#creation of list of all agents
ag_list = []
att = att
var = var
obj = obj #define attributes, variables, and objective for each agent individually
ag_list.append(Agent(att, var, obj))
# due to the fact that agents are both much less prolific than nodes, and often require specific
# initial placement, each agent is append to the list seperately

continuing = True
while continuing:
    #go through all bots and nodes, updating to the next time step
    for bot in ag_list:
        bot.sensing(env_list, ag_list)
        bot.processing()
    for point in env_list:
        point.update()
    for bot in ag_list:
        bot.actuating()

    if conditions == True:
        continuing = False
        #if certain conditions are met, end the loop running the simulation


# if rendering in post , a copy of each agents self.X and the variables of each node could be saved at
# each time step, and exported to be run in a seperate function.
    



ModuleNotFoundError: No module named 'environment'

In [6]:
import pygame
import numpy as np
import math

class Node:
    def __init__(self, x, y, vars=None):
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (optional)

    def update(self):
        # Update logic for the node (if needed)
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the node (grid point)
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)

class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = attributes  # External aspects (e.g., position, size)
        self.V = variables   # Internal aspects (e.g., memory, pressure)
        self.O = objective   # Objectives (e.g., target position)
        self.E = [[], []]    # Local environment (nodes and agents)

    def sensing(self, environment, agents):
        # Senses the local environment (e.g., nodes and agents in proximity)
        self.E = [environment, agents]

    def processing(self):
        # Processes the sensed data and updates internal state
        pass

    def actuating(self):
        # Updates the external state (e.g., position) based on internal state
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the agent on the screen
        pygame.draw.circle(surface, color,
                           (int(self.X[0] * scale_factor + offset_x),
                            int(self.X[1] * scale_factor + offset_y)),
                           10)  # Agents drawn as larger circles

class Environment:
    def __init__(self, x_range, y_range, spacing):
        self.coarse_nodes = self.construct_grid(x_range, y_range, spacing)
        self.min_x, self.min_y = self.coarse_nodes[0].position  # Top-left corner
        self.max_x, self.max_y = self.coarse_nodes[-1].position  # Bottom-right corner

    def construct_grid(self, x_range, y_range, spacing):
        """ 
        Creates an array of nodes within the specified x and y ranges, 
        with a defined distance between all nodes.
        """
        environment = []
        i = 0
        while x_range[0] + i * spacing <= x_range[1]:
            j = 0
            while y_range[0] + j * spacing <= y_range[1]:
                var_specific = None  # Modify this as needed for custom variables
                environment.append(Node(x_range[0] + i * spacing, y_range[0] + j * spacing, var_specific))
                j += 1
            i += 1
        return environment

    def generate_fine_grid_circular(self, selected_node, fine_spacing, radius):
        """ Generate a finer grid around a selected node within a circular region """
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2

        # Define the range to search for points in the circular region
        range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
        range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

        for i in range_x:
            for j in range_y:
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                # Calculate distance from the selected node
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

                # Check if the point is within the circular region and within the coarse boundary
                if distance_squared <= radius_squared and self.min_x <= x <= self.max_x and self.min_y <= y <= self.max_y:
                    nodes.append(Node(x, y))
        return nodes

    def generate_fine_grid_regular(self, selected_node, range_units, fine_spacing):
        """ Generate a regular finer grid around a selected node """
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                # Check to ensure the fine grid does not exceed the coarse grid boundaries
                if x <= self.max_x and y <= self.max_y:
                    nodes.append(Node(x, y))
        return nodes

    def refine_grid(self, selected_node, refine_type, fine_spacing, radius, range_units):
        if refine_type == 'circular':
            return self.generate_fine_grid_circular(selected_node, fine_spacing, radius)
        elif refine_type == 'regular':
            return self.generate_fine_grid_regular(selected_node, range_units, fine_spacing)
        else:
            return []

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents and Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60  # Set a higher FPS for smoother rendering

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1  # Coarse grid spacing (1 unit)

# Initialize the environment with the coarse grid
environment = Environment(x_range, y_range, spacing)

# Calculate dynamic scaling factor so that the grid fits within the window
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])  # 100 pixels margin
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])  # 100 pixels margin
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (environment.coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (environment.coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

fine_nodes = []
for node, refine_type in nodes_to_refine:
    fine_nodes.extend(environment.refine_grid(node, refine_type, fine_spacing=0.5, radius=2, range_units=2))

# Agent initialization (define agent attributes, variables, and objectives)
agents = [
    Agent([2.5, 3.5], {"memory": 0}, {"target": [5, 5]}),  # Example agent at position (2, 3)
    Agent([6.5, 5.5], {"memory": 0}, {"target": [9, 9]}),  # Example agent at position (6, 5)
    Agent([7.5, 6.5], {"memory": 0}, {"target": [10, 10]}) # Example agent at position (8, 9)
]

# Main simulation loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a simple white background

    # Go through all bots and nodes, updating to the next time step
    for bot in agents:
        bot.sensing(environment.coarse_nodes, agents)  # Agent senses environment
        bot.processing()  # Agent processes information

    for point in environment.coarse_nodes:
        point.update()  # Update the environment (nodes)
        point.draw(window, (0, 0, 255), scaling_factor, offset_x, offset_y)  # Blue nodes

    for point in fine_nodes:
        point.draw(window, (255, 0, 0), scaling_factor, offset_x, offset_y)  # Red refined nodes

    for bot in agents:
        bot.actuating()  # Agent actuates based on processing
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Green agents

    pygame.display.flip()  # Update the screen
    clock.tick(fps)  # Maintain the frame rate

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Quit Pygame
pygame.quit()


In [9]:
import pygame
import numpy as np

class Node:
    def __init__(self, x, y, vars=None):
        # Initialize the position attribute directly from the input x, y coordinates
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (optional)

    def update(self):
        # Update logic for the node (if needed)
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the node (grid point) based on its stored position
        pygame.draw.circle(surface, color,
                           (int(self.position[0] * scale_factor + offset_x),
                            int(self.position[1] * scale_factor + offset_y)),
                           5)

    @staticmethod
    def construct_grid(x_range, y_range, spacing):
        """Creates a grid of nodes within specified x and y ranges, spaced evenly by 'spacing'."""
        grid_nodes = []
        # Iterate to generate nodes within the range using position
        for x in np.arange(x_range[0], x_range[1] + spacing, spacing):
            for y in np.arange(y_range[0], y_range[1] + spacing, spacing):
                # Create each node and initialize its position
                node = Node(x, y)
                grid_nodes.append(node)
        return grid_nodes

    @staticmethod
    def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
        """Generate a finer grid around a selected node within a circular region."""
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2

        # Define the range to search for points in the circular region
        range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
        range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

        for i in range_x:
            for j in range_y:
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                # Calculate distance from the selected node
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2

                # Check if the point is within the circular region and within the boundary
                if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

    @staticmethod
    def generate_fine_grid_regular(selected_node, range_units, fine_spacing, max_x, max_y):
        """Generate a regular finer grid around a selected node."""
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                # Check to ensure the fine grid does not exceed the boundaries
                if x <= max_x and y <= max_y:
                    nodes.append(Node(x, y))
        return nodes


# Additional example code for visualization or testing is similar to the previous snippet
# and omitted here for brevity.

class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = attributes  # External aspects (e.g., position, size)
        self.V = variables   # Internal aspects (e.g., memory, pressure)
        self.O = objective   # Objectives (e.g., target position)
        self.E = [[], []]    # Local environment (nodes and agents)

    def sensing(self, environment, agents):
        # Senses the local environment (e.g., nodes and agents in proximity)
        self.E = [environment, agents]

    def processing(self):
        # Processes the sensed data and updates internal state
        pass

    def actuating(self):
        # Updates the external state (e.g., position) based on internal state
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        # Draw the agent on the screen
        pygame.draw.circle(surface, color,
                           (int(self.X[0] * scale_factor + offset_x),
                            int(self.X[1] * scale_factor + offset_y)),
                           10)  # Agents drawn as larger circles
# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500  # Set the size of the window
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents and Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60  # Set a higher FPS for smoother rendering

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1  # Coarse grid spacing (1 unit)

# Initialize the environment with the coarse grid
environment = Environment(x_range, y_range, spacing)

# Calculate dynamic scaling factor so that the grid fits within the window
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])  # 100 pixels margin
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])  # 100 pixels margin
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (environment.coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (environment.coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

fine_nodes = []
for node, refine_type in nodes_to_refine:
    fine_nodes.extend(environment.refine_grid(node, refine_type, fine_spacing=0.5, radius=2, range_units=2))

# Agent initialization (define agent attributes, variables, and objectives)
agents = [
    Agent([2.5, 3.5], {"memory": 0}, {"target": [5, 5]}),  # Example agent at position (2, 3)
    Agent([2.5, 5.5], {"memory": 0}, {"target": [9, 9]}),  # Example agent at position (6, 5)
    Agent([7.5, 6.5], {"memory": 0}, {"target": [10, 10]}) # Example agent at position (8, 9)
]

# Main simulation loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a simple white background

    # Go through all bots and nodes, updating to the next time step
    for bot in agents:
        bot.sensing(environment.coarse_nodes, agents)  # Agent senses environment
        bot.processing()  # Agent processes information

    for point in environment.coarse_nodes:
        point.update()  # Update the environment (nodes)
        point.draw(window, (0, 0, 255), scaling_factor, offset_x, offset_y)  # Blue nodes

    for point in fine_nodes:
        point.draw(window, (255, 0, 0), scaling_factor, offset_x, offset_y)  # Red refined nodes

    for bot in agents:
        bot.actuating()  # Agent actuates based on processing
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Green agents

    pygame.display.flip()  # Update the screen
    clock.tick(fps)  # Maintain the frame rate

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Quit Pygame
pygame.quit()


In [14]:
import pygame
import numpy as np

class Node:
    def __init__(self, x, y, vars=None, is_obstacle=False):
        # Initialize the position and obstacle flag
        self.position = np.array([float(x), float(y)])  # Sets the position of the node
        self.vars = vars  # Any additional variables (optional)
        self.is_obstacle = is_obstacle  # Flag to mark if this node is an obstacle

    def update(self):
        # Update logic for the node (if needed)
        pass

    def draw(self, surface, color, obstacle_color, scale_factor, offset_x, offset_y):
        # Draw obstacle nodes in a different color
        if self.is_obstacle:
            pygame.draw.circle(surface, obstacle_color,
                               (int(self.position[0] * scale_factor + offset_x),
                                int(self.position[1] * scale_factor + offset_y)),
                               5)
        else:
            pygame.draw.circle(surface, color,
                               (int(self.position[0] * scale_factor + offset_x),
                                int(self.position[1] * scale_factor + offset_y)),
                               5)

    @staticmethod
    def construct_grid(x_range, y_range, spacing):
        """Creates a grid of nodes within specified x and y ranges, spaced evenly by 'spacing'."""
        grid_nodes = []
        for x in np.arange(x_range[0], x_range[1] + spacing, spacing):
            for y in np.arange(y_range[0], y_range[1] + spacing, spacing):
                node = Node(x, y)
                grid_nodes.append(node)
        return grid_nodes

    @staticmethod
    def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
        """Generate a finer grid around a selected node within a circular region."""
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2

        range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
        range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

        for i in range_x:
            for j in range_y:
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2
                if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

    @staticmethod
    def generate_fine_grid_regular(selected_node, range_units, fine_spacing, max_x, max_y):
        """Generate a regular finer grid around a selected node."""
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                if x <= max_x and y <= max_y:
                    nodes.append(Node(x, y))
        return nodes


class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = attributes  # External aspects (e.g., position, size)
        self.V = variables   # Internal aspects (e.g., memory, pressure)
        self.O = objective   # Objectives (e.g., target position)
        self.E = [[], []]    # Local environment (nodes and agents)

    def sensing(self, environment, agents):
        self.E = [environment, agents]

    def processing(self):
        pass

    def actuating(self):
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        pygame.draw.circle(surface, color,
                           (int(self.X[0] * scale_factor + offset_x),
                            int(self.X[1] * scale_factor + offset_y)),
                           10)  # Agents drawn as larger circles

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents and Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1

# Initialize the environment with the coarse grid and mark some nodes as obstacles
environment = Environment(x_range, y_range, spacing)
obstacle_positions = [(3, 3), (5, 5), (7, 7)]  # Specify obstacle positions

for point in environment.coarse_nodes:
    if tuple(point.position) in obstacle_positions:
        point.is_obstacle = True  # Mark these nodes as obstacles

# Calculate dynamic scaling factor
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Initialize agents
agents = [
    Agent([2.5, 3.5], {"memory": 0}, {"target": [5, 5]}),
    Agent([2.5, 5.5], {"memory": 0}, {"target": [9, 9]}),
    Agent([7.5, 6.5], {"memory": 0}, {"target": [10, 10]})
]

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (environment.coarse_nodes[0], 'circular'),  # Node 0 refined circularly
    (environment.coarse_nodes[5], 'regular'),   # Node 5 refined regularly
]

# Generate fine nodes for each refinement type
fine_nodes = []
for node, refine_type in nodes_to_refine:
    if refine_type == 'circular':
        fine_nodes.extend(Node.generate_fine_grid_circular(node, fine_spacing=0.5, radius=2, min_x=0, max_x=10, min_y=0, max_y=10))
    elif refine_type == 'regular':
        fine_nodes.extend(Node.generate_fine_grid_regular(node, range_units=2, fine_spacing=0.5, max_x=10, max_y=10))

# Main simulation loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a white background

    # Go through all bots and nodes, updating to the next time step
    for bot in agents:
        bot.sensing(environment.coarse_nodes, agents)
        bot.processing()

    for point in environment.coarse_nodes:
        point.update()
        point.draw(window, (0, 0, 255), (128, 128, 128), scaling_factor, offset_x, offset_y)  # Blue nodes, Grey obstacles

    for point in fine_nodes:
        point.draw(window, (255, 0, 0), (128, 128, 128), scaling_factor, offset_x, offset_y)  # Red refined nodes

    for bot in agents:
        bot.actuating()
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Green agents

    pygame.display.flip()
    clock.tick(fps)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

# Quit Pygame
pygame.quit()


NameError: name 'Environment' is not defined

In [13]:
import pygame
import numpy as np

class Node:
    def __init__(self, x, y, vars=None, is_obstacle=False):
        self.position = np.array([float(x), float(y)])
        self.vars = vars
        self.is_obstacle = is_obstacle

    def update(self):
        pass

    def draw(self, surface, color, obstacle_color, scale_factor, offset_x, offset_y):
        if self.is_obstacle:
            pygame.draw.circle(surface, obstacle_color,
                               (int(self.position[0] * scale_factor + offset_x),
                                int(self.position[1] * scale_factor + offset_y)),
                               5)
        else:
            pygame.draw.circle(surface, color,
                               (int(self.position[0] * scale_factor + offset_x),
                                int(self.position[1] * scale_factor + offset_y)),
                               5)

    @staticmethod
    def construct_grid(x_range, y_range, spacing):
        grid_nodes = []
        for x in np.arange(x_range[0], x_range[1] + spacing, spacing):
            for y in np.arange(y_range[0], y_range[1] + spacing, spacing):
                node = Node(x, y)
                grid_nodes.append(node)
        return grid_nodes

    @staticmethod
    def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2
        range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
        range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

        for i in range_x:
            for j in range_y:
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2
                if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

    @staticmethod
    def generate_fine_grid_regular(selected_node, range_units, fine_spacing, max_x, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                if x <= max_x and y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

class Agent:
    def __init__(self, attributes, variables, objective):
        
        # Unpack the `attributes` list into individual agent properties
        self.X = {
            "ID": attributes[0],                  # Unique identifier for the agent
            "position": np.array(attributes[1]),  # Position as a 2D or 3D vector
            "radius": attributes[2],              # Radius representing the agent's size in the environment
            "vision": attributes[3],              # Vision range for detecting neighboring agents
            "status": attributes[4],              # Role or function of the agent in the system
            "angle": attributes[5] if len(attributes) > 5 else 0,  # Orientation; defaults to 0 if not provided
            "speed": attributes[6] if len(attributes) > 6 else 0   # Speed; defaults to 0 if not provided
        }

        # Initialize dynamic internal variables
        self.V = variables if variables else {
            "neighbor_agents": [],               # List of neighboring agents within vision radius
            "statuses": {},                      # Statuses of neighboring agents by ID
            "neighbor_ids": []                   # IDs of neighboring agents
        }

        # Objective and local environment
        self.O = objective                       # Objective or target information
        self.E = [] 

        # Initialize dynamic internal state variables
        self.V['neighbor_agents'] = []  # List of neighboring agents within vision radius
        self.V['statuses'] = {}         # Statuses of neighboring agents by ID
        self.V['neighbor_ids'] = []     # IDs of neighboring agents

    def sensing(self, coarse_grid, fine_grid, agents):
        """Detects nodes in both coarse and fine grids, and agents within vision radius."""
        vision_radius = 2
        vision_angle = 360  # Full circular vision
        self.E = []  # Reset environment perception list
        
    
        # Clear dynamic internal variables at the start of sensing
        self.V['neighbor_agents'] = []
        self.V['statuses'] = {}
        self.V['neighbor_ids'] = []

        # Detect nodes within the vision radius from coarse grid
        for node in coarse_grid:
            distance = np.linalg.norm(self.X - node.position)
            if distance <= vision_radius:
                detected_info = {
                    "type": "node",
                    "position": node.position,
                    "distance": distance,
                    "is_obstacle": node.is_obstacle
                }
                self.E.append(detected_info)

        # Detect nodes within the vision radius from fine grid
        for node in fine_grid:
            distance = np.linalg.norm(self.X - node.position)
            if distance <= vision_radius:
                detected_info = {
                    "type": "fine_node",
                    "position": node.position,
                    "distance": distance,
                    "is_obstacle": node.is_obstacle
                }
                self.E.append(detected_info)

        # Detect other agents within the vision radius
        for other_agent in agents:
            if other_agent != self:
                distance = np.linalg.norm(self.X - other_agent.X)
                if distance <= vision_radius:
                    detected_info = {
                        "type": "agent",
                        "position": other_agent.X,
                        "distance": distance
                    }
                    self.E.append(detected_info)

    def processing(self):
        pass

    def actuating(self):
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        pygame.draw.circle(surface, color,
                           (int(self.X[0] * scale_factor + offset_x),
                            int(self.X[1] * scale_factor + offset_y)),
                           10)  # Agents drawn as larger circles

# Initialize Pygame
pygame.init()

# Set up the display
width, height = 500, 500
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Node Grid with Agents and Refinement")

# Set the clock for controlling frame rate
clock = pygame.time.Clock()
fps = 60

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1
# Initialize the environment with the coarse grid and mark some nodes as obstacles
environment = Environment(x_range, y_range, spacing)
# # Initialize the environment with the coarse grid and mark some nodes as obstacles
# coarse_grid = Environment.construct_grid(x_range, y_range, spacing)
obstacle_positions = [(3, 3), (5, 5), (7, 7)]  # Specify obstacle positions

for point in environment:
    if tuple(point.position) in obstacle_positions:
        point.is_obstacle = True  # Mark these nodes as obstacles

# Calculate dynamic scaling factor
scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])
scaling_factor = min(scaling_factor_x, scaling_factor_y)

# Calculate offset to center the grid within the window
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

# Initialize agents
agents = [
    Agent([2.5, 3.5], {"memory": 0}, {"target": [5, 5]}),
    Agent([2.5, 5.5], {"memory": 0}, {"target": [9, 9]}),
    Agent([7.5, 6.5], {"memory": 0}, {"target": [10, 10]})
]

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (environment[0], 'circular'),  # Node 0 refined circularly
    (environment[5], 'regular'),   # Node 5 refined regularly
]

# Generate fine nodes for each refinement type
fine_grid = []
for node, refine_type in nodes_to_refine:
    if refine_type == 'circular':
        fine_grid.extend(Node.generate_fine_grid_circular(node, fine_spacing=0.5, radius=2, min_x=0, max_x=10, min_y=0, max_y=10))
    elif refine_type == 'regular':
        fine_grid.extend(Node.generate_fine_grid_regular(node, range_units=2, fine_spacing=0.5, max_x=10, max_y=10))

# Main simulation loop
running = True
while running:
    window.fill((255, 255, 255))  # Clear screen with a white background

    # Agent senses environment
    for bot in agents:
        bot.sensing(coarse_grid, fine_grid, agents)
        bot.processing()

    # Draw coarse grid nodes and obstacles
    for point in coarse_grid:
        point.update()
        point.draw(window, (0, 0, 255), (128, 128, 128), scaling_factor, offset_x, offset_y)  # Blue nodes, Grey obstacles

    # Draw refined (fine grid) nodes
    for point in fine_grid:
        point.draw(window, (255, 0, 0), (128, 128, 128), scaling_factor, offset_x, offset_y)  # Red refined nodes

    # Draw agents
    for bot in agents:
        bot.actuating()
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)  # Green agents

    # Print sensed objects for each agent
    for bot in agents:
        print("Agent at position:", bot.X)
        print("Detected objects within vision:")
        for detected_object in bot.E:
            print(detected_object)
        print("------")

    pygame.display.flip()
    clock.tick(fps)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()


NameError: name 'Environment' is not defined

In [16]:
import pygame
import numpy as np

class Node:
    def __init__(self, x, y, vars=None, is_obstacle=False):
        self.position = np.array([float(x), float(y)])
        self.vars = vars
        self.is_obstacle = is_obstacle

    def update(self):
        pass

    def draw(self, surface, color, obstacle_color, scale_factor, offset_x, offset_y):
        pos = (int(self.position[0] * scale_factor + offset_x),
               int(self.position[1] * scale_factor + offset_y))
        pygame.draw.circle(surface, obstacle_color if self.is_obstacle else color, pos, 5)

    @staticmethod
    def construct_grid(x_range, y_range, spacing):
        grid_nodes = []
        for x in np.arange(x_range[0], x_range[1] + spacing, spacing):
            for y in np.arange(y_range[0], y_range[1] + spacing, spacing):
                grid_nodes.append(Node(x, y))
        return grid_nodes

    @staticmethod
    def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2
        for i in range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1):
            for j in range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2
                if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

    @staticmethod
    def generate_fine_grid_regular(selected_node, range_units, fine_spacing, max_x, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                if x <= max_x and y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = {
            "ID": attributes[0],
            "position": np.array(attributes[1]),
            "radius": attributes[2],
            "vision": attributes[3],
            "status": attributes[4],
            "angle": attributes[5] if len(attributes) > 5 else 0,
            "speed": attributes[6] if len(attributes) > 6 else 0
        }
        self.V = variables if variables else {
            "neighbor_agents": [],
            "statuses": {},
            "neighbor_ids": []
        }
        self.O = objective
        self.E = []
        self.refined_grid = []  # Store refined grid nodes

    def sensing(self, coarse_grid, agents, refine_type='none'):
        """Detects nodes within vision and refines them based on refine_type."""
        vision_radius = self.X["vision"]
        self.E = []  # Reset environment perception list
        self.refined_grid = []  # Reset refined grid nodes

        # Detect nodes within vision radius
        for node in coarse_grid:
            distance = np.linalg.norm(self.X["position"] - node.position)
            if distance <= vision_radius:
                self.E.append({
                    "type": "node",
                    "position": node.position,
                    "distance": distance,
                    "is_obstacle": node.is_obstacle
                })

                # Apply fine grid refinement if specified
                if refine_type == 'circular':
                    self.refined_grid.extend(Node.generate_fine_grid_circular(
                        node, fine_spacing=0.5, radius=vision_radius, min_x=0, max_x=10, min_y=0, max_y=10
                    ))
                elif refine_type == 'regular':
                    self.refined_grid.extend(Node.generate_fine_grid_regular(
                        node, range_units=2, fine_spacing=0.5, max_x=10, max_y=10
                    ))

        # Detect other agents within vision radius
        for other_agent in agents:
            if other_agent != self:
                distance = np.linalg.norm(self.X["position"] - other_agent.X["position"])
                if distance <= vision_radius:
                    self.E.append({
                        "type": "agent",
                        "position": other_agent.X["position"],
                        "distance": distance,
                        "status": other_agent.X["status"],
                        "ID": other_agent.X["ID"]
                    })
                    self.V["neighbor_agents"].append(other_agent)
                    self.V["statuses"][other_agent.X["ID"]] = other_agent.X["status"]
                    self.V["neighbor_ids"].append(other_agent.X["ID"])

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        agent_pos = (int(self.X["position"][0] * scale_factor + offset_x),
                     int(self.X["position"][1] * scale_factor + offset_y))
        pygame.draw.circle(surface, color, agent_pos, 10)

        # Draw vision radius circle
        pygame.draw.circle(surface, (0, 255, 255), agent_pos, int(self.X["vision"] * scale_factor), 1)

        # Draw refined grid nodes if they exist
        for node in self.refined_grid:
            node.draw(surface, (255, 0, 0), (128, 128, 128), scale_factor, offset_x, offset_y)

# Initialize Pygame and environment setup
pygame.init()
width, height = 500, 500
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Agent Vision and Grid Refinement")

clock = pygame.time.Clock()
fps = 60

x_range = (0, 10)
y_range = (0, 10)
spacing = 1

coarse_grid = Node.construct_grid(x_range, y_range, spacing)

scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])
scaling_factor = min(scaling_factor_x, scaling_factor_y)
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

agents = [
    Agent([1, [2.5, 3.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]}),
    Agent([1, [2.5, 5.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]}),
    Agent([1, [7.5, 6.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]})
]



# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (coarse_grid[0], 'circular'),  # Node 0 refined circularly
    (coarse_grid[5], 'regular'),   # Node 5 refined regularly
]

# Generate fine nodes for each refinement type
fine_nodes = []
for node, refine_type in nodes_to_refine:
    if refine_type == 'circular':
        fine_nodes.extend(Node.generate_fine_grid_circular(node, fine_spacing=0.5, radius=2, min_x=0, max_x=10, min_y=0, max_y=10))
    elif refine_type == 'regular':
        fine_nodes.extend(Node.generate_fine_grid_regular(node, range_units=2, fine_spacing=0.5, max_x=10, max_y=10))

running = True
while running:
    window.fill((255, 255, 255))

    # Agents sense environment with fine grid refinement (circular or regular)
    for bot in agents:
        bot.sensing(coarse_grid, agents, refine_type='circular')  # Set to 'circular' or 'regular'
    
    # Draw nodes
    for point in coarse_grid:
        point.update()
        point.draw(window, (0, 0, 255), (128, 128, 128), scaling_factor, offset_x, offset_y)

    # Draw agents and refined grids
    for bot in agents:
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)

    pygame.display.flip()
    clock.tick(fps)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()


SyntaxError: invalid syntax (268609292.py, line 150)

In [24]:
import pygame
import numpy as np

class Node:
    def __init__(self, x, y, vars=None, is_obstacle=False):
        self.position = np.array([float(x), float(y)])
        self.vars = vars
        self.is_obstacle = is_obstacle

    def update(self):
        pass

    def draw(self, surface, color, obstacle_color, scale_factor, offset_x, offset_y):
        pos = (int(self.position[0] * scale_factor + offset_x),
               int(self.position[1] * scale_factor + offset_y))
        pygame.draw.circle(surface, obstacle_color if self.is_obstacle else color, pos, 5)

    @staticmethod
    def construct_grid(x_range, y_range, spacing):
        grid_nodes = []
        for x in np.arange(x_range[0], x_range[1] + spacing, spacing):
            for y in np.arange(y_range[0], y_range[1] + spacing, spacing):
                grid_nodes.append(Node(x, y))
        return grid_nodes

    @staticmethod
    def generate_fine_grid_circular(selected_node, fine_spacing, radius, min_x, max_x, min_y, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        radius_squared = radius ** 2
        range_x = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)
        range_y = range(-int(radius / fine_spacing), int(radius / fine_spacing) + 1)

        for i in range_x:
            for j in range_y:
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                distance_squared = (x - selected_node_x) ** 2 + (y - selected_node_y) ** 2
                if distance_squared <= radius_squared and min_x <= x <= max_x and min_y <= y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

    @staticmethod
    def generate_fine_grid_regular(selected_node, range_units, fine_spacing, max_x, max_y):
        nodes = []
        selected_node_x, selected_node_y = selected_node.position
        for i in range(int(range_units / fine_spacing) + 1):
            for j in range(int(range_units / fine_spacing) + 1):
                x = selected_node_x + j * fine_spacing
                y = selected_node_y + i * fine_spacing
                if x <= max_x and y <= max_y:
                    nodes.append(Node(x, y))
        return nodes

class Agent:
    def __init__(self, attributes, variables, objective):
        self.X = {
            "ID": attributes[0],
            "position": np.array(attributes[1]),
            "radius": attributes[2],
            "vision": attributes[3],
            "status": attributes[4],
            "angle": attributes[5] if len(attributes) > 5 else 0,
            "speed": attributes[6] if len(attributes) > 6 else 0
        }
        self.V = variables if variables else {
            "neighbor_agents": [],
            "statuses": {},
            "neighbor_ids": []
        }
        self.O = objective
        self.E = []
        self.refined_grid = []  # Store refined grid nodes

    def sensing(self, entities):
            """Detects nodes, obstacles, and other agents within the vision radius."""
            vision_radius = self.X["vision"]
            self.E = []  # Reset environment perception list
            self.V['neighbor_agents'] = []
            self.V['statuses'] = {}
            self.V['neighbor_ids'] = []

            # Loop through all entities (nodes and agents) within the vision radius
            for entity in entities:
                # Determine position based on whether entity is a Node or Agent
                if isinstance(entity, Node):
                    position = entity.position
                elif isinstance(entity, Agent) and entity != self:
                    position = entity.X["position"]
                else:
                    continue  # Skip if it's the same agent or an unknown entity type

                # Calculate distance from the agent to the entity
                distance = np.linalg.norm(self.X["position"] - position)
                if distance <= vision_radius:
                    if isinstance(entity, Node):
                        detected_info = {
                            "type": "obstacle" if entity.is_obstacle else "node",
                            "position": entity.position,
                            "distance": distance,
                            "is_obstacle": entity.is_obstacle
                        }
                    elif isinstance(entity, Agent):
                        detected_info = {
                            "type": "agent",
                            "position": entity.X["position"],
                            "distance": distance,
                            "status": entity.X["status"],
                            "ID": entity.X["ID"]
                        }
                        # Update dynamic internal variables for agents
                        self.V["neighbor_agents"].append(entity)
                        self.V["statuses"][entity.X["ID"]] = entity.X["status"]
                        self.V["neighbor_ids"].append(entity.X["ID"])

                    # Add detected information to the environment perception list
                    self.E.append(detected_info)

    def processing(self):
        pass

    def actuating(self):
        pass

    def draw(self, surface, color, scale_factor, offset_x, offset_y):
        agent_pos = (int(self.X["position"][0] * scale_factor + offset_x),
                     int(self.X["position"][1] * scale_factor + offset_y))
        pygame.draw.circle(surface, color, agent_pos, 10)

        # Draw vision radius circle
        pygame.draw.circle(surface, (0, 255, 255), agent_pos, int(self.X["vision"] * scale_factor), 1)

        # Draw refined grid nodes if they exist
        for node in self.refined_grid:
            node.draw(surface, (255, 0, 0), (128, 128, 128), scale_factor, offset_x, offset_y)

# Initialize Pygame and environment setup
pygame.init()
width, height = 500, 500
window = pygame.display.set_mode((width, height))
pygame.display.set_caption("Agent Vision and Grid Refinement")

clock = pygame.time.Clock()
fps = 60

# Define grid settings
x_range = (0, 10)
y_range = (0, 10)
spacing = 1

# Create coarse grid and mark some nodes as obstacles
coarse_grid = Node.construct_grid(x_range, y_range, spacing)
obstacle_positions = [(3, 3), (5, 5), (7, 7)]
for point in coarse_grid:
    if tuple(point.position) in obstacle_positions:
        point.is_obstacle = True

scaling_factor_x = (width - 100) // (x_range[1] - x_range[0])
scaling_factor_y = (height - 100) // (y_range[1] - y_range[0])
scaling_factor = min(scaling_factor_x, scaling_factor_y)
offset_x = (width - ((x_range[1] - x_range[0]) * scaling_factor)) // 2
offset_y = (height - ((y_range[1] - y_range[0]) * scaling_factor)) // 2

agents = [
    Agent([1, [2.5, 3.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]}),
    Agent([2, [3.5, 4.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]}),
    Agent([3, [7.5, 6.5], 0.5, 2, "active", 45], {"memory": 0}, {"target": [5, 5]})
]

# Define nodes to refine and their respective refinement types
nodes_to_refine = [
    (coarse_grid[0], 'circular'),  # Node 0 refined circularly
    (coarse_grid[5], 'regular'),   # Node 5 refined regularly
]

# Generate fine nodes for each refinement type
fine_grid = []
for node, refine_type in nodes_to_refine:
    if refine_type == 'circular':
        fine_grid.extend(Node.generate_fine_grid_circular(node, fine_spacing=0.5, radius=2, min_x=0, max_x=10, min_y=0, max_y=10))
    elif refine_type == 'regular':
        fine_grid.extend(Node.generate_fine_grid_regular(node, range_units=2, fine_spacing=0.5, max_x=10, max_y=10))

running = True

while running:
    window.fill((255, 255, 255))  # Clear screen with a white background

    # Combine all entities (nodes and agents) into a single list for sensing
    all_entities = coarse_grid + fine_grid + agents

    # Agent senses environment using the combined list of entities
    for bot in agents:
        bot.sensing(all_entities)  # Pass the combined list to sensing

    # Print sensed information for each agent
    for bot in agents:
        print(f"Agent ID: {bot.X['ID']} at Position: {bot.X['position']}")
        print("Sensed Entities:")
        for detected in bot.E:
            if detected["type"] == "agent":
                print(f" - Agent ID {detected['ID']} at Position {detected['position']}, Distance: {detected['distance']}")
            elif detected["type"] == "node":
                print(f" - Node at Position {detected['position']}, Distance: {detected['distance']}")
            elif detected["type"] == "obstacle":
                print(f" - Obstacle at Position {detected['position']}, Distance: {detected['distance']}")
        print("------")  # Separator for readability between agents
    
    # Draw nodes
    for point in coarse_grid:
        point.update()
        point.draw(window, (0, 0, 255), (128, 128, 128), scaling_factor, offset_x, offset_y)

    # Draw refined (fine grid) nodes
    for point in fine_grid:
        point.draw(window, (255, 0, 0), (128, 128, 128), scaling_factor, offset_x, offset_y)

    # Draw agents
    for bot in agents:
        bot.draw(window, (0, 255, 0), scaling_factor, offset_x, offset_y)

    pygame.display.flip()
    clock.tick(fps)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()



Agent ID: 1 at Position: [2.5 3.5]
Sensed Entities:
 - Node at Position [1. 3.], Distance: 1.5811388300841898
 - Node at Position [1. 4.], Distance: 1.5811388300841898
 - Node at Position [2. 2.], Distance: 1.5811388300841898
 - Node at Position [2. 3.], Distance: 0.7071067811865476
 - Node at Position [2. 4.], Distance: 0.7071067811865476
 - Node at Position [2. 5.], Distance: 1.5811388300841898
 - Node at Position [3. 2.], Distance: 1.5811388300841898
 - Obstacle at Position [3. 3.], Distance: 0.7071067811865476
 - Node at Position [3. 4.], Distance: 0.7071067811865476
 - Node at Position [3. 5.], Distance: 1.5811388300841898
 - Node at Position [4. 3.], Distance: 1.5811388300841898
 - Node at Position [4. 4.], Distance: 1.5811388300841898
 - Node at Position [1.5 5. ], Distance: 1.8027756377319946
 - Node at Position [2. 5.], Distance: 1.5811388300841898
 - Agent ID 2 at Position [3.5 4.5], Distance: 1.4142135623730951
------
Agent ID: 2 at Position: [3.5 4.5]
Sensed Entities:
 - No

: 